In [1]:
import nltk, unicodedata
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import time
import glob
import errno
import pandas as pd
from sklearn import naive_bayes, metrics, preprocessing
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('stopwords')
from gensim.models import Phrases
from gensim import corpora, models


[nltk_data] Downloading package stopwords to C:\Users\Laptop
[nltk_data]     Ku\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def casefolding(text):
    text = text.lower()
    return text

def word_token(text):
    tokens = word_tokenize(text)

    return tokens

def stemming (tokens):
    porter_stemmer = PorterStemmer()
    result = list()
    for word in tokens:
        result.append(porter_stemmer.stem(word))
        
    return ' '.join(result)
    return text

def stopword (tokens):
    stop_words = set(stopwords.words('english'))
    clean_text = []
    for word in tokens:
        if word not in stop_words:
            clean_text.append(word)
    return clean_text

def noise_rem (text):
    text = re.sub('[^\w\s]','', text)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = re.sub('[\s]+', ' ', text)
    return text

def bersih2 (baca):
    baca = casefolding(baca)
    baca = noise_rem(baca)
    tok = word_token(baca)
    tok = stopword(tok)
    tok = stemming(tok)
    baca = ''.join(tok)
    
    return(baca)

a = ("there is something weird that happened recently while i was cleaning my room. the closet's door was opened by itself even though i lock it")
b = bersih2(a)
print (b)

someth weird happen recent clean room closet door open even though lock


In [19]:
import xlsxwriter

        
def masukdata (path):
    start_time = time.time()
    files = glob.glob(path)
    texts = []
    workbook = xlsxwriter.Workbook('clean-data.xlsx')
    worksheet = workbook.add_worksheet()

    row = 0
    col = 0

    rowHeaders = ['text']
    worksheet.write_row(row, col,  tuple(rowHeaders))
    for review in files:
        f = open(review,'r', encoding='utf-8')
        baca = f.read()
        f.close()
        
        baca = bersih2(baca)
        rowValues = [baca]
        row += 1
        worksheet.write_row(row, col, tuple(rowValues))

    workbook.close()
#         texts.append(baca)
#     trainDF = pd.DataFrame()
#     trainDF['text'] = texts
    print("--- %s seconds ---" % (time.time() - start_time))

sets = masukdata('./unsup/*.txt')
print("done")

--- 14.497674226760864 seconds ---
done


In [20]:
fo = pd.ExcelFile('clean-data.xlsx')
df = pd.read_excel(fo, 'Sheet1')

text = df['text']
print(type(df))
print(df)
text_list =  [i.split() for i in text]
print(len(text_list))

<class 'pandas.core.frame.DataFrame'>
                                                   text
0     admit great major film releas say 1933 dozen m...
1     say name like zombiegeddon atom bomb front cov...
2     want read spoiler read thank youbr br ill hone...
3     see movi expect budget bad good zombi flick sa...
4     bad bone sicknessbr br well yeah isbr br movi ...
5     movi isnt worth much time total crap rent movi...
6     fan zombiemovi one disappoint movi without goo...
7     okay feel must prais given film bad rap deserv...
8     film geek could get anyth piec garabag meant s...
9     marvel movi start finishbr br meet robert mont...
10    base broadway play anoth languag one gem yeste...
11    movi must rereleas whole gener comedi fan miss...
12    anyon interest famili relationship sure enjoy ...
13    kati tippel similar turkish delight film ton n...
14    movi easi see director made holland land mains...
15    record movi week ago local commun televis stat...
16    like

In [21]:
#Create Biagram & Trigram Models 
from gensim.models import Phrases
# Add bigrams and trigrams to docs,minimum count 10 means only that appear 10 times or more.
bigram = Phrases(text_list, min_count=10)
trigram = Phrases(bigram[text_list], min_count=10)

for idx in range(len(text_list)):
    for token in bigram[text_list[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            text_list[idx].append(token)
    for token in trigram[text_list[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            text_list[idx].append(token)

C:\ProgramData\Anaconda3\lib\site-packages\gensim\models\phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [22]:
from gensim import corpora, models
# Create a dictionary representation of the documents.
dictionary = corpora.Dictionary(text_list)
dictionary.filter_extremes(no_below=5, no_above=0.2)
print(dictionary)

Dictionary(7873 unique tokens: ['1933', 'actor', 'admit', 'appreci', 'apprehens']...)


In [23]:
#build corpus
doc_term_matrix = [dictionary.doc2bow(doc) for doc in text_list]

print(len(doc_term_matrix))
print(doc_term_matrix[100])

tfidf = models.TfidfModel(doc_term_matrix)
corpus_tfidf = tfidf[doc_term_matrix]

3586
[(33, 1), (67, 1), (112, 1), (217, 1), (345, 1), (505, 1), (652, 1), (836, 1), (879, 1), (900, 1), (1419, 3), (1616, 1), (1717, 3), (1857, 1), (2503, 2), (2508, 2), (2509, 3), (2612, 1), (2613, 1), (2614, 1), (2615, 1), (2616, 1), (2617, 1), (2618, 1)]


In [67]:
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.models.lsimodel import LsiModel
from gensim.corpora.dictionary import Dictionary
from numpy import array
#function to compute coherence values
def compute_coherence_values(dictionary, corpus, texts, limit, start, step, mdl):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        if(mdl == 0):
            model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, iterations=100)
            model_list.append(model)
            coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
            coherence_values.append(coherencemodel.get_coherence())
        else:
            model = LsiModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, 
                             onepass=False, power_iters=150)
            model_list.append(model)
            coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
            coherence_values.append(coherencemodel.get_coherence())
        
    return model_list, coherence_values

In [ ]:
start=2
limit=22
step=2

start_time = time.time()
model_list, coherence_values = compute_coherence_values(dictionary, corpus=corpus_tfidf, 
                                                        texts=text_list, start=start, limit=limit, step=step,mdl = 0)
print("LDA done")
print("--- %s seconds ---" % (time.time() - start_time))

model_list1, coherence_values1 = compute_coherence_values(dictionary, corpus=corpus_tfidf, 
                                                        texts=text_list, start=start, limit=limit, step=step, mdl=1)
print("LSI done")
print("--- %s seconds ---" % (time.time() - start_time))

#show graphs
import matplotlib.pyplot as plt
print("LDA model")
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

print("LSI model")
x = range(start, limit, step)
plt.plot(x, coherence_values1)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()
print("--- %s seconds ---" % (time.time() - start_time))

LDA done
--- 187.1713547706604 seconds ---


In [56]:
# Print the coherence scores
print ('LDA model')
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))
    
print ('LSI model')
for m, cv in zip(x, coherence_values1):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

LDA model
Num Topics = 2  has Coherence Value of 0.2853
Num Topics = 4  has Coherence Value of 0.2949
Num Topics = 6  has Coherence Value of 0.3935
Num Topics = 8  has Coherence Value of 0.404
Num Topics = 10  has Coherence Value of 0.3949
Num Topics = 12  has Coherence Value of 0.4358
Num Topics = 14  has Coherence Value of 0.4719
Num Topics = 16  has Coherence Value of 0.4568
Num Topics = 18  has Coherence Value of 0.5326
Num Topics = 20  has Coherence Value of 0.5032
LSI model
Num Topics = 2  has Coherence Value of 0.349
Num Topics = 4  has Coherence Value of 0.3031
Num Topics = 6  has Coherence Value of 0.3745
Num Topics = 8  has Coherence Value of 0.3697
Num Topics = 10  has Coherence Value of 0.338
Num Topics = 12  has Coherence Value of 0.3389
Num Topics = 14  has Coherence Value of 0.3291
Num Topics = 16  has Coherence Value of 0.3378
Num Topics = 18  has Coherence Value of 0.3591
Num Topics = 20  has Coherence Value of 0.3663


In [58]:
from pprint import pprint

print('LDA')
model = LdaModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=18, iterations=100)
pprint(model.print_topics())

print()
print('LSI')

model1 = LsiModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=6, onepass=True, power_iters=100)
pprint(model1.print_topics())

LDA
[(0,
  '0.006*"bill_murray" + 0.003*"juic" + 0.003*"ava_gardner" + '
  '0.003*"comic_relief" + 0.003*"soap_opera" + 0.002*"pull_punch" + '
  '0.002*"one_best" + 0.002*"jen" + 0.002*"murray" + 0.002*"comput_gener"'),
 (1,
  '0.004*"hoffman" + 0.002*"make_sens" + 0.002*"etc_etc" + '
  '0.002*"agatha_christi" + 0.002*"huge_fan" + 0.002*"havent_seen" + '
  '0.002*"21st_centuri" + 0.002*"ever_made" + 0.002*"timebr_br" + '
  '0.002*"ever_sinc"'),
 (2,
  '0.005*"we_anderson" + 0.005*"sherlock_holm" + 0.004*"christma" + '
  '0.003*"sherlock" + 0.003*"walk_around" + 0.002*"cassidi" + '
  '0.002*"even_though" + 0.002*"quentin_tarantino" + 0.002*"church" + '
  '0.002*"releas_dvd"'),
 (3,
  '0.004*"stori_line" + 0.003*"special_effect" + 0.003*"realli_enjoy" + '
  '0.003*"year_ago" + 0.003*"ginger" + 0.003*"dragon" + 0.003*"ginger_roger" + '
  '0.002*"everi_time" + 0.002*"havent_seen" + 0.002*"fight_club"'),
 (4,
  '0.003*"worth_look" + 0.003*"old_man" + 0.003*"serial_killer" + '
  '0.003*"end_

In [52]:
model = LdaModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=18, iterations=100)
model1 = LsiModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=6, onepass=True, power_iters=100)

for idx, topic in model.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.003*"hoffman" + 0.003*"dont_miss" + 0.003*"monk" + 0.003*"90_minut" + 0.002*"main_charact" + 0.002*"one_greatest" + 0.002*"look_like" + 0.002*"pretti_good" + 0.002*"ginger" + 0.002*"even_though"
Topic: 1 Word: 0.006*"we_anderson" + 0.005*"mother_sister" + 0.003*"act_superb" + 0.003*"definit_worth" + 0.003*"dustin" + 0.003*"lenni" + 0.003*"10_minut" + 0.003*"havent_seen" + 0.003*"video_store" + 0.002*"lenni_bruce"
Topic: 2 Word: 0.008*"de_niro" + 0.006*"werewolf" + 0.004*"philip_k" + 0.003*"would_recommend" + 0.003*"camera_work" + 0.003*"niro" + 0.003*"real_life" + 0.003*"well_done" + 0.003*"norway" + 0.002*"tyron_power"
Topic: 3 Word: 0.005*"morgan_freeman" + 0.003*"your_look" + 0.003*"fight_club" + 0.003*"freeman" + 0.003*"agatha_christi" + 0.003*"star_war" + 0.002*"one_best" + 0.002*"edg_seat" + 0.002*"nativ" + 0.002*"main_charact"
Topic: 4 Word: 0.003*"king_kong" + 0.002*"ten_minut" + 0.002*"want_see" + 0.002*"make_sens" + 0.002*"reilli" + 0.002*"bad" + 0.002*"point

In [53]:
top_words_per_topic = []
for t in range(model.num_topics):
    top_words_per_topic.extend([(t, ) + x for x in model.show_topic(t, topn = 18)])

#pd.DataFrame(top_words_per_topic, columns=['Topic', 'Word', 'P']).to_csv("top_words.csv")
df = pd.DataFrame(top_words_per_topic, columns=['Topic', 'Word','P']).to_csv("top_words2.csv")
print(df)

None


In [66]:
import gensim
import pyLDAvis.gensim;pyLDAvis.enable_notebook();

data = pyLDAvis.gensim.prepare(model, corpus_tfidf, dictionary)
print(data)
pyLDAvis.save_html(data, 'lda-gensim-18.html')

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13     0.099029  0.053613       1        1  24.643002
17     0.045927 -0.005586       2        1  10.992599
15     0.038881 -0.035311       3        1   9.106997
14     0.030978 -0.084428       4        1   7.899704
3      0.015495 -0.034375       5        1   6.660711
1      0.036429  0.028984       6        1   6.179914
10    -0.010607 -0.000022       7        1   3.982857
11    -0.019669  0.014217       8        1   3.587323
2     -0.007402  0.008293       9        1   3.558315
5     -0.013541 -0.032758      10        1   3.252084
6     -0.007907  0.021920      11        1   3.155763
8     -0.043170  0.027007      12        1   3.065491
0     -0.028183  0.006334      13        1   2.864668
4     -0.008914  0.010435      14        1   2.800848
7     -0.014697  0.032247      15        1   2.231612
9     -0.038979 -0.003706      16        1   2.1067